In [1]:
import random
from math import inf


params = {
    "farmer_productivity": 5.0,
    "land_capacity": 80,
    "steal_rate": 2.0,
    "suppression_prob": 0.4,     # p_hit
    "confiscate_rate": 1.5,
    "tax_rate": 0.10,
    "soldier_base_pay": 1.0,

    "alpha_perception": 0.4,
    "beta_tolerance": 0.1,
    "fear_increment": 0.5,
    "fear_decay": 0.2,

    "utility_margin": 0.2,       # margin
    "switch_urge_threshold": 0.5 # urge_threshold
}

EPS = 1e-6
FEAR_MAX = 10.0

class Person:
    def __init__(self, role, wealth=0.0, prod_mult=1.0, aggr_mult=1.0, skill_mult=1.0):
        self.role = role
        self.wealth = max(0.0, wealth)

        self.fear = 0.0
        self.perceived_violence = 0.0
        self.loyalty = 1.0

        self.next_role = role
        self.age_in_role = 0
        self.switch_cooldown = 0  # optional churn control

        self.last_production = 0
        self.last_rob_gain = 0
        self.last_pay = 0

        # simple heterogeneity
        self.prod_mult = prod_mult
        self.aggr_mult = aggr_mult
        self.skill_mult = skill_mult

    # --- housekeeping ---
    def reset_for_new_tick(self):
        self.next_role = self.role
        self.age_in_role += 1
        if self.switch_cooldown > 0:
            self.switch_cooldown -= 1

    # --- role helpers ---
    def is_farmer(self):  return self.role == "farmer"
    def is_bandit(self):  return self.role == "bandit"
    def is_soldier(self): return self.role == "soldier"

    # --- actions ---
    def produce(self, farmer_productivity, scale=1.0):
        if not self.is_farmer(): return 0.0
        inc = max(0.0, self.prod_mult * farmer_productivity * max(0.0, scale))
        self.wealth = max(0.0, self.wealth + inc)
        self.last_production = inc
        return inc

    def attempt_rob(self, victim, steal_rate, fear_bump_robbed=1.0):
        if not (self.is_bandit() and victim and victim.role == "farmer"):
            return 0.0, False
        stolen = min(max(0.0, self.aggr_mult * steal_rate), max(0.0, victim.wealth))
        if stolen <= 0: return 0.0, False
        victim.wealth = max(0.0, victim.wealth - stolen)
        victim.fear = min(FEAR_MAX, victim.fear + max(0.0, fear_bump_robbed))
        self.wealth = max(0.0, self.wealth + stolen)
        self.last_rob_gain = stolen
        return stolen, True

    def attempt_suppress(self, target_bandit, p_hit, confiscate_rate, fear_bump_suppressed=1.0):
        if not (self.is_soldier() and target_bandit and target_bandit.role == "bandit"):
            return 0.0, False
        if random.random() >= max(0.0, min(1.0, self.skill_mult * p_hit)):
            return 0.0, False
        amt = min(max(0.0, self.skill_mult * confiscate_rate), max(0.0, target_bandit.wealth))
        # count as violent even if amt==0 (still coercion)
        target_bandit.wealth = max(0.0, target_bandit.wealth - amt)
        if fear_bump_suppressed > 0:
            target_bandit.fear = min(FEAR_MAX, target_bandit.fear + fear_bump_suppressed)
        self.wealth = max(0.0, self.wealth + amt)
        return amt, True

    def pay_tax(self, tax_rate):
        if not self.is_farmer(): return 0.0
        tax = max(0.0, tax_rate) * max(0.0, self.wealth)
        self.wealth = max(0.0, self.wealth - tax)
        return tax

    def receive_pay(self, amount):
        if amount > 0: self.wealth = max(0.0, self.wealth + amount)
        self.last_pay = amount

    # --- psychology ---
    def update_psychology(self, global_violence, alpha=0.4, fear_inc=0.5, fear_dec=0.2):
        a = max(0.0, min(1.0, alpha))
        gv = max(0.0, global_violence)
        self.perceived_violence = (1 - a) * self.perceived_violence + a * gv
        if self.perceived_violence > 0.5:
            self.fear = min(FEAR_MAX, self.fear + max(0.0, fear_inc))
        else:
            self.fear = max(0.0, self.fear - max(0.0, fear_dec))

    # --- switching ---
    def decide_switch(self, loyalty_value, utilities, switch_urge_threshold, utility_margin):
        if utilities is None:
            utilities = {}
        
        if self.switch_cooldown > 0:
            self.next_role = self.role
            return
        self.loyalty = max(EPS, loyalty_value)
        urge = self.fear / (self.loyalty + EPS)
        if urge <= switch_urge_threshold:
            self.next_role = self.role
            return
        current_u = utilities.get(self.role, -1e9)
        best_role, best_u = self.role, current_u
        for r, u in utilities.items():
            if r != self.role and u > best_u:
                best_role, best_u = r, u
        self.next_role = best_role if best_u > current_u + utility_margin else self.role

    def finalize_role_switch(self):
        if self.next_role != self.role:
            self.role = self.next_role
            self.age_in_role = 0
            self.switch_cooldown = 1  # small hysteresis (optional)

    # --- export ---
    def to_dict(self):
        return {
            "role": self.role,
            "wealth": round(self.wealth, 3),
            "fear": self.fear,
            "perceived_violence": self.perceived_violence,
            "loyalty": self.loyalty,
            "age_in_role": self.age_in_role,
        }

In [39]:
class World:
    def __init__(self, n_farmers, n_bandits, n_soilders):

        #initial setup
        self.people = []
        self.farmer_pop = n_farmers
        self.bandit_pop = n_bandits
        self.soiler_pop = n_bandits

        for i in range(n_farmers):
            self.people.append(Person('farmer'))
        for i in range(n_bandits):
            self.people.append(Person('bandit'))
        for i in range(n_soilders):
            self.people.append(Person('soldier'))
        
        self.update_population_counts()

        # global vmetrics
        self.total_robberies = 0
        self.total_suppressions = 0

        self.utilities_for_roles = {
            "farmer": 0.0,
            "bandit": 0.0,
            "soldier": 0.0
        }

    # Utility: count role populations
    def update_population_counts(self):
        self.farmer_pop = sum(p.is_farmer() for p in self.people)
        self.bandit_pop = sum(p.is_bandit() for p in self.people)
        self.soldier_pop = sum(p.is_soldier() for p in self.people)

    # Main tick update function, will run once per iteration
    def step(self):
        for person in self.people:
            person.reset_for_new_tick()

        # Production: farmers produce wealth
        for person in self.people:
            if person.is_farmer():
                person.produce(farmer_productivity=.15)

        # Robberies: bandits attempt to steal from farmers
        farmers = [p for p in self.people if p.is_farmer()]
        bandits = [p for p in self.people if p.is_bandit()] # 
        random.shuffle(bandits)

        self.total_robberies = 0

        for bandit in bandits:
            if not farmers:
                break
            target = random.choice(farmers)
            stole_amount = bandit.attempt_rob(target, steal_rate=0.2)
            if stole_amount[0] > 0:
                self.total_robberies += 1

        # Suppressions: Soldiers suppress bandits
        soldiers = [p for p in self.people if p.is_soldier()]
        bandits = [p for p in self.people if p.is_bandit()]

        self.total_suppressions = 0

        for soldier in soldiers:
            if not bandits:
                break
            target = random.choice(bandits)
            hit = soldier.attempt_suppress(target, p_hit = 0.6, confiscate_rate=2.5)
            if hit:
                self.total_suppressions += 1

        # Tax Farmers and Pay Soldiers
        tax_pool = 0.0
        for farmer in farmers:
            tax_pool += farmer.pay_tax(tax_rate=0.5)

        if soldiers:
            pay_per_soldier = tax_pool / len(soldiers)
        else:
            pay_per_soldier = 0.0

        for soldier in soldiers:
            soldier.receive_pay(pay_per_soldier)

        # Calculating utilities
        self.expected_prod_per_farmer = (sum(p.last_production for p in farmers) / len(farmers) if farmers else 0.0)
        self.expected_rob_per_bandit = (sum(p.last_rob_gain for p in bandits) / len(bandits) if bandits else 0.0)
        self.expected_soldier_pay = (sum(p.last_pay for p in soldiers) / len(soldiers) if soldiers else 0.0)

        self.utilities_for_roles = {
            "farmer": self.expected_prod_per_farmer,
            "bandit": self.expected_rob_per_bandit,
            "soldier": self.expected_soldier_pay
        }

        # Psychological updates
        for person in self.people:
            person.update_psychology(global_violence=self.total_robberies + self.total_suppressions)

        # Role-switch decisions
        for person in self.people:
            person.decide_switch(loyalty_value=person.loyalty, utilities=self.utilities_for_roles, switch_urge_threshold=0.3, utility_margin=0.2)

        # Finalize switching
        for person in self.people:
            person.finalize_role_switch()

        # update population counts
        self.update_population_counts()

    # Export world state (for logging or data collection)
    def to_dict(self):
        return {
            "farmer_pop": self.farmer_pop,
            "bandit_pop": self.bandit_pop,
            "soldier_pop": self.soldier_pop,
            "total_robberies": self.total_robberies,
            "total_suppressions": self.total_suppressions,
            "utilities": self.utilities_for_roles
        }

In [64]:
my_world = World(50, 50, 50)
my_world.to_dict()

{'farmer_pop': 50,
 'bandit_pop': 50,
 'soldier_pop': 50,
 'total_robberies': 0,
 'total_suppressions': 0,
 'utilities': {'farmer': 0.0, 'bandit': 0.0, 'soldier': 0.0}}

In [80]:
my_world.step()
my_world.to_dict()

{'farmer_pop': 50,
 'bandit_pop': 50,
 'soldier_pop': 50,
 'total_robberies': 38,
 'total_suppressions': 50,
 'utilities': {'farmer': 0.15,
  'bandit': 0.155609375,
  'soldier': 0.03191992187499999}}